In [45]:
import random
import warnings

import gunzip
import wget

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import Bio
import BioSQL
from Bio import SeqIO
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import csv
#parse file, clean data right after
GENOME_FILE = "C:/Users/pixil/PycharmProjects/SimpleTorchProject\TorchDun/data/nih_human_gproj/Homo_sapiens.GRCh38.dna.primary_assembly.fa"
#Final and constant, unless we change it before running ofc
#FINAL
FEATURE_FILE = "C:/Users/pixil/PycharmProjects/SimpleTorchProject/TorchDun/data/nih_human_gproj/GCF_000001405.40_GRCh38.p14_feature_table.txt"
SEQUENCE_LENGTH = 1000  # Change sequence length
NUM_SAMPLES = 10000     # Change number of sequences
OUTPUT_FILE = "data/sequences.csv"

df_feature = pd.read_csv(
    FEATURE_FILE,
    sep='\t',
    comment='#',
    header=None,
    usecols=[0, 1, 5, 7, 8, 9, 13, 14, 15, 19],
    #NCIB database named
    names=['feature', 'class', 'chromosome', 'start', 'end', "strand", "description", "symbol", "GeneID", "attributes"]
)
#our features
print(df_feature.columns.tolist())
def extract_sequences(GENOME_FILE, SEQUENCE_LENGTH, NUM_SAMPLES):
    seq = []
    for record in SeqIO.parse(GENOME_FILE, "fasta"):
        chrom_name = record.id
        chrom_seq = str(record.seq).upper() #string of sequence (letters)
        chrom_len = len(chrom_seq) #gets our length of sequence
        #gets all genes on the chromosome
        chrom_genes = df_feature[(df_feature['chromosome'] == chrom_name) & (df_feature['feature'] == 'gene')]
        print(record.id)
        print(f"Processing chromosome {chrom_name}: {chrom_len:,} bp")
        for i in range(NUM_SAMPLES):
            if len(seq) >= NUM_SAMPLES:
                break
            start = random.randint(0, chrom_len - SEQUENCE_LENGTH)
            chunk = chrom_seq[start:start + SEQUENCE_LENGTH]

            #we have to check if gene is in sequence!
            gene_snip = ((chrom_genes['start'] < start + SEQUENCE_LENGTH) & (chrom_genes['end'] > start ))
            in_gene = gene_snip.any()
            if in_gene:
                gene_in_seq = chrom_genes[gene_snip]
                gene_name = ','.join(gene_in_seq['symbol'].dropna().tolist())
            else:
                gene_name = ''
            #gets bases count
            g_count = chunk.count('G')
            c_count = chunk.count('C')
            a_count = chunk.count('A')
            t_count = chunk.count('T')

            #we have refseq and genseq categories as well i believe

            #percent in count
            gc_count = ((g_count + c_count) / SEQUENCE_LENGTH) * 100
            at_count = ((a_count + t_count) / SEQUENCE_LENGTH) * 100
            seq.append({
                "sequence": chunk,
                "chromosome": chrom_name,
                "start": start,
                "end": start + SEQUENCE_LENGTH,
                "gc_count": gc_count,
                "at_count": at_count,
                "in_gene": in_gene,
                "gene_name": gene_name
            })
        if len(seq) >= NUM_SAMPLES:
            break

    return seq
sequences = extract_sequences(GENOME_FILE, SEQUENCE_LENGTH, NUM_SAMPLES)

df = pd.DataFrame(sequences)
#sequences
print(df)
#data now must be cleaned and put in a dataframe, set up for training+analysis



['feature', 'class', 'chromosome', 'start', 'end', 'strand', 'description', 'symbol', 'GeneID', 'attributes']
1
Processing chromosome 1: 248,956,422 bp
                                               sequence chromosome      start  \
0     AGAGCTTCAGGGAAGGAGAACCCCAGAGATCTGTAGAGGGTATTCC...          1  221576856   
1     TAAAATTAAAAGTATAGAAATAGAAACAAAGCAATAACATTCAAGT...          1  157370272   
2     TATTTTTTGAAACGTTGGGCAATGTCTGACTTTAGCATCTGTGTTA...          1  184341761   
3     TTTTTGTGTATCCAATTATAGACCCTAATATCTAATTGCAGATTCT...          1  232467920   
4     CCCCTCCCCACAGCCCCTGGCAACCACAAATCTACTTTCTGTCTCT...          1  109164203   
...                                                 ...        ...        ...   
9995  ATTTATAATTCATTGAGGTAAGCACACAGGGACGAAGGAGTCAGCC...          1  236687338   
9996  GTTAATTATTACCAATAGAAGCTTATTTATGCTTGTAACTGAGTTA...          1  243365017   
9997  ACGCCATTCTCCTGCTTCAGCCTCCCGAGTAGCTGGGACTACAGGT...          1   10372728   
9998  GGTCTGCGATCTGAGGTCTTGCTTTAGCAAGC